In [2]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import glob
import numpy as np
import rasterio
import rasterio.mask
from os.path import exists
from shutil import copyfile
from pathlib import Path

#### Step 1: set up basic folders

In [3]:
def create_folder(name):
    try:
        os.mkdir(name)
    except FileExistsError:
        pass

In [4]:
create_folder('data') # folder for raw data
create_folder('shapefile')
create_folder('output') # folder for processed data
create_folder('output/drought')
create_folder('plots') # folder for plots
create_folder('stats') # folder for derived statistics
create_folder('stats/drought')
create_folder('stats/CCKP')
create_folder('stats/spei')
create_folder('maps') # folder for maps

country = os.getcwd().split('\\')[-1]

# Create corresponding folders on an external hard drive to store large data. Change file path as needed.
create_folder('F:/World Bank/City Scan/' + country)
create_folder('F:/World Bank/City Scan/' + country + '/data')
create_folder('F:/World Bank/City Scan/' + country + '/output')

#### Step 2: create AOI shapefiles
Have the AOI shapefiles in the shapefile folder before proceeding
The AOI shapefiles should be named such as new_york.shp (lower case, spaces replaced with underscores)

#### Step 3: project AOI shapefiles into WGS 84 and save them to city folders

In [4]:
cities = ['Kinshasa', 'Kananga', 'Mbuji-Mayi', 'Lubumbashi', 'Kisangani', 'Bukavu', 'Goma', 'Tshikapa', 'Mwene-Ditu', 'Gemena', 'Gbadolite', 'Matadi', 'Kikwit', 'Bunia']

In [5]:
for city in cities:
    create_folder(Path(city))
    create_folder(city / Path('maps'))
    create_folder(city / Path('data'))
    create_folder(city / Path('data') / 'AOI')
    create_folder(city / Path('stats'))

In [6]:
for city in cities:
    shp_4326 = gpd.read_file(Path('shapefile') / (city.replace(" ", "_").lower() + '.shp')).to_crs(epsg = 4326)
    shp_4326.to_file(Path(city) / 'data/AOI' / (city.replace(" ", "_").lower() + '_AOI.shp'))

C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.

#### Step 4: generate city coordinates and UTM

In [7]:
with open('centroids.csv', 'w') as f:
    f.write('city,x,y,utm\n')
    for city in cities:
        centroid = gpd.read_file(Path(city) / 'data/AOI' / (city.replace(" ", "_").lower() + '_AOI.shp')).centroid
        f.write('%s,%s,%s,%s\n'%(city, centroid.x[0], centroid.y[0], 32600+math.ceil((centroid.x[0]+180)/6)))

C:\Users\Owner\AppData\Local\Temp\ipykernel_19860\2829273085.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gpd.read_file(Path(city) / 'data/AOI' / (city.replace(" ", "_").lower() + '_AOI.shp')).centroid
C:\Users\Owner\AppData\Local\Temp\ipykernel_19860\2829273085.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gpd.read_file(Path(city) / 'data/AOI' / (city.replace(" ", "_").lower() + '_AOI.shp')).centroid
C:\Users\Owner\AppData\Local\Temp\ipykernel_19860\2829273085.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gpd.read_file

#### Step 5: extract country shapefile

In [9]:
# WB countries shapefile. Change file path as needed
all_countries = gpd.read_file(r"C:\Users\Owner\Documents\Career\World Bank\City Scan\data\wb_countries_admin0_10m\WB_countries_Admin0_10m.shp")

In [14]:
# change country name as needed
country_shp = all_countries[all_countries.NAME_EN == 'Democratic Republic of the Congo']

In [13]:
country_shp

,OBJECTID,featurecla,LEVEL,TYPE,FORMAL_EN,FORMAL_FR,POP_EST,POP_RANK,GDP_MD_EST,POP_YEAR,...,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,WB_NAME,WB_RULES,WB_REGION,Shape_Leng,Shape_Area,geometry
29,30,Admin-0 country,2,Sovereign country,Democratic Republic of the Congo,None,83301151,16,66010.0,2017,...,Kongo-Kinshasa,Demokratik Kongo Cumhuriyeti,Cộng hòa Dân chủ Congo,刚果民主共和国,"Congo, Democratic Republic of","Always have ""Democratic Republic of"" before ""C...",AFR,87.70156,189.678904,"MULTIPOLYGON (((18.62639 3.47687, 18.59300 3.7..."


In [15]:
country_shp.to_file(Path('shapefile') / (country.lower().replace(' ', '_') + '.shp'))

C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Step 6: make centroids shapefile

In [16]:
centroids = pd.read_csv('centroids.csv')

In [17]:
gdf = gpd.GeoDataFrame(centroids, 
                       geometry = gpd.points_from_xy(centroids.x, centroids.y))

C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [18]:
gdf.to_file('shapefile/centroids.shp', crs = 'EPSG:4326')

C:\Users\Owner\anaconda3\envs\cityscan\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Step 7: copy GEE outputs to city output folders

In [19]:
def crop_gee_output(city, raster_name):
    with rasterio.open(Path('output') / 'GEE' / (raster_name + '.tif')) as src:
        array = src.read(1)
        out_meta = src.meta.copy()
        out_meta.update({'nodata': 'nan'})
    with rasterio.open(city / Path('data') / (raster_name + '.tif'), 'w', **out_meta) as dest:
        dest.write(array, 1)

In [20]:
for city in cities:
    crop_gee_output(city, city.replace(" ", "_").lower() + '_Summer')

#### Step 8: copy files needed for Match Map Style toolbox

In [21]:
fathom_folder = Path('data') / 'Fathom'
floods = {'fluvial_undefended': 'FU',
          'pluvial': 'P'}
rps = [5, 10, 20, 50, 75, 100, 200, 250, 500, 1000]

In [23]:
for city in cities:
    for flood in floods:
        for rp in rps:
            copyfile(fathom_folder / flood / (floods[flood] + '_1in' + str(rp) + '.tif'), 
                     city / Path('data') / (floods[flood] + '_1in' + str(rp) + '.tif'))

#### Step 9: delete files needed for Match Map Style toolbox after maps are produced

In [5]:
for city in cities:
    for flood in floods:
        for rp in rps:
            os.remove(city / Path('data') / (floods[flood] + '_1in' + str(rp) + '.tif'))